# Artificial Message Generation Using Large Language Models

This notebook demonstrates how to generate artificial messages (campaign) based on tweets using different LLMs. The process involves fetching the original text of a tweet, and then using a prompt to generate a new, artificial tweet that follows the theme or content of the original.


# A todo list. 
Snopes

Understand all the api for more naunced manipulation 

# Data 
## Twitter....

Import necessary libraries

Get Tweets/data

Twitter not working. Need paid account. Data more


In [1]:
import os
from dotenv import load_dotenv
import tweepy
import pandas as pd


# Load Twitter API credentials from environment variables
auth = tweepy.OAuthHandler(os.getenv('TWITTER_CONSUMER_KEY'), os.getenv('TWITTER_CONSUMER_SECRET'))
auth.set_access_token(os.getenv('TWITTER_ACCESS_TOKEN'), os.getenv('TWITTER_ACCESS_SECRET'))
twitter_api = tweepy.API(auth)


# To do subjectivity and polarity
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

# Current data

subject to change and experiment

In [39]:
tweet="Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy"

#Doable, can adjust the prompts accordingly 
# Define global prompts that can be adjusted accordingly
CONTROL_PROMPT = "Generate a closely related tweet that subtly varies in wording but retains the essence and context of the original tweet: {}"
EMOTIONAL_PROMPT = "Rephrase the following tweet to evoke a {} feeling: {}"
EMOTIONAL_COT_PROMPT = ("Rephrase the following tweet to evoke a {} feeling by thinking step-by-step: {}. "
                        "Let's think step-by-step to evoke {}. First, identify the key elements of the tweet. "
                        "Next, modify the language to enhance {}. Finally, ensure the tweet effectively conveys {}:"
                        "Return only one tweet to me.")
CONSPIRACY_PROMPT = "Rewrite the tweet to sound like a conspiracy theory: {}"
ADVERSARIAL_PROMPT_POSITIVE = "Given the tweet '{}', craft a response that challenges the statement, aiming to provoke or polarize the audience while staying within ethical guidelines."
ADVERSARIAL_PROMPT_NEGATIVE = "Given the tweet '{}', craft a response that agrees with and intensifies the negative sentiment, aiming to provoke or polarize the audience while staying within ethical guidelines."


def analyze_text(text):
    doc = nlp(text)
    return {
        "text": text,
        "polarity": doc._.blob.polarity,
        "subjectivity": doc._.blob.subjectivity
    }


# Model 1: Chat GPT

understand all parameters

More parameters to be aware of:

- top_p (float, optional): Controls the nucleus sampling where the model considers the smallest set of words whose cumulative probability exceeds the probability p. This helps in focusing the generation on more likely outcomes.
- frequency_penalty (float, optional): Adjusts the likelihood of the model repeating the same line verbatim, with higher values discouraging repetition.
- presence_penalty (float, optional): Adjusts the likelihood of the model repeating phrases, with higher values encouraging the introduction of new concepts.



Polarity measures the emotional content of the text, ranging from -1 (very negative) to +1 (very positive).

It essentially indicates the sentiment tone of the text based on the adjectives used.





Conspiracy Tweet Function

This function is designed to generate tweets that contain elements of misinformation. Misinformation often stems from or aligns with personal beliefs rather than established facts (need a reference to back this claim up). This connection is crucial because it highlights the subjective nature of the content typically found in conspiracy theories.

Measured by Subjectivity.
Subjectivity quantifies how much of the text is based on personal opinions, emotions, or judgments versus factual information. The scale ranges from 0 (very objective) to 1 (very subjective).



Adversarial Tweet Function

Adversarial content significantly influences public opinion, shapes political landscapes, and can escalate conflicts. Extensive research has explored techniques to "jailbreak" large language models (LLMs) or conduct universal and transferable adversarial attacks on aligned language models, prompting GPT to generate contentious content.

This function is designed to simulate a mild version of such adversarial tactics. It aims to test the boundaries of content generation without severely breaching ethical or operational constraints imposed by advanced LLMs. As language models continue to evolve, their defenses improve, making it increasingly challenging to generate genuinely harmful or hateful content without detection.

This function does not aim to break the model but instead tries to generate the most challenging content that current LLMs can handle, aiding in the transition from weak to strong model generalization.


In [28]:
from openai import OpenAI

class ChatGPTHandler:
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.nlp = nlp

    def _make_request(self, prompt, model="gpt-3.5-turbo-instruct", temperature=1, max_tokens=256):
        try:
            response = self.client.completions.create(
                model=model,
                prompt=prompt,
                temperature=temperature,
                max_tokens=max_tokens
            )
            generated_text = response.choices[0].text.strip()
            analysis = self.analyze_text(generated_text)
            return {
                "generated_text": generated_text,
                "polarity": analysis["polarity"],
                "subjectivity": analysis["subjectivity"]
            }
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        prompt = CONTROL_PROMPT.format(tweet)
        return self._make_request(prompt)

    def emotional_tweet(self, tweet_text, emotion):
        prompt = EMOTIONAL_PROMPT.format(emotion, tweet_text)
        return self._make_request(prompt)

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        prompt = EMOTIONAL_COT_PROMPT.format(emotion, tweet_text, emotion, emotion, emotion)
        return self._make_request(prompt)

    def conspiracy_tweet(self, tweet_text):
        prompt = CONSPIRACY_PROMPT.format(tweet_text)
        return self._make_request(prompt)

    def adversarial_tweet(self, tweet_text):
        original_analysis = self.analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            prompt = ADVERSARIAL_PROMPT_POSITIVE.format(tweet_text)
        else:
            prompt = ADVERSARIAL_PROMPT_NEGATIVE.format(tweet_text)
        return self._make_request(prompt)


Example Usage

In [31]:
api_key = os.getenv('OPENAI_API_KEY')
tweet_handler = ChatGPTHandler(api_key)

original_analysis = analyze_text(tweet)
print("Original Tweet Analysis:", original_analysis)

control_result = tweet_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = tweet_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = tweet_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = tweet_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = tweet_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Original Tweet Analysis: {'text': 'Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy', 'polarity': 0.30333333333333334, 'subjectivity': 0.7383333333333333}
Control Tweet: {'generated_text': '"Thrilled to hear that #renewableenergy use is skyrocketing and making a major impact in lowering carbon emissions. Time to keep pushing for #ClimateAction and embrace sustainable #GreenEnergy solutions!"', 'polarity': 0.3390625, 'subjectivity': 0.6}
Emotional Tweet: {'generated_text': '"Fantastic news! This year, we reached record-breaking levels of renewable energy use, resulting in a remarkable reduction in carbon emissions! 💚 #ClimateAction #GreenEnergy"', 'polarity': 0.71875, 'subjectivity': 0.825}
Emotional CoT Tweet: {'generated_text': "Fantastic update! This year, the use of renewable energy has skyrocketed, resulting in a major drop in carbon emissions. It's an incredible achievement fo

# Model 2: Gemini

Need to understand more on the api and exact parameters it takes

In [58]:
import google.generativeai as genai


class GeminiHandler:
    def __init__(self, api_key, model_name="gemini-1.5-flash"):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name=model_name)
        self.nlp = nlp

    def _make_request(self, prompt):
        try:
            response = self.model.generate_content([prompt])
            generated_text = response.text
            analysis = self._analyze_text(generated_text)
            return {
                "generated_text": generated_text,
                "polarity": analysis["polarity"],
                "subjectivity": analysis["subjectivity"]
            }
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        prompt = CONTROL_PROMPT.format(tweet)
        return self._make_request(prompt)

    def emotional_tweet(self, tweet_text, emotion):
        prompt = EMOTIONAL_PROMPT.format(emotion, tweet_text)
        return self._make_request(prompt)

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        prompt = EMOTIONAL_COT_PROMPT.format(emotion, tweet_text, emotion, emotion, emotion)
        return self._make_request(prompt)

    def conspiracy_tweet(self, tweet_text):
        prompt = CONSPIRACY_PROMPT.format(tweet_text)
        return self._make_request(prompt)

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            prompt = ADVERSARIAL_PROMPT_POSITIVE.format(tweet_text)
        else:
            prompt = ADVERSARIAL_PROMPT_NEGATIVE.format(tweet_text)
        return self._make_request(prompt)


In [60]:
gemini_handler = GeminiHandler(api_key=os.getenv("GEMINI_API"))


control_result = gemini_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = gemini_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = gemini_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = gemini_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = gemini_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: {'generated_text': "Great news for our planet! Renewable energy is booming, driving a major reduction in carbon emissions. Let's keep pushing for a cleaner future. #ClimateChange #Sustainability \n", 'polarity': 0.3541666666666667, 'subjectivity': 0.4583333333333333}
Emotional Tweet: {'generated_text': "🎉  Good news, everyone!  Renewable energy is booming this year, making a huge difference in cutting down on harmful emissions.  We're on the right track to a cleaner, brighter future!  #ClimateAction #GreenEnergy 💚 \n", 'polarity': 0.13419312169312173, 'subjectivity': 0.5582671957671957}
Emotional CoT Tweet: {'generated_text': "🎉 Good news, everyone!  Renewable energy is booming this year, making a big difference in reducing carbon emissions. We're making progress towards a healthier planet, one clean energy source at a time!  #ClimateAction #GreenEnergy \n", 'polarity': 0.4444444444444445, 'subjectivity': 0.46666666666666673}
Conspiracy Tweet: {'generated_text': 'They\'r

# Model 3 Llama

Streaming vs non streaming, look up and see whats going on.


In [50]:

from llamaapi import LlamaAPI
import json

llama = LlamaAPI(os.getenv('LLAMA_API'))

class LlamaHandler:
    def __init__(self, api_key, model_name="llama3-70b"):
        self.llama = LlamaAPI(api_key)
        self.model_name = model_name
        self.nlp = nlp

    def _make_request(self, prompt):
        api_request_json = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": "You are a tweeter assistant that rephrases tweets well, return only the tweet without further ado"},
                {"role": "user", "content": prompt},
            ]
        }
        try:
            response = self.llama.run(api_request_json)
            generated_text = response.json()['choices'][0]['message']['content']
            analysis = self._analyze_text(generated_text)
            return {
                "generated_text": generated_text,
                "polarity": analysis["polarity"],
                "subjectivity": analysis["subjectivity"]
            }
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        prompt = CONTROL_PROMPT.format(tweet)
        return self._make_request(prompt)
    
    def emotional_tweet(self, tweet_text, emotion):
        prompt = EMOTIONAL_PROMPT.format(emotion, tweet_text)
        return self._make_request(prompt)

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        prompt = EMOTIONAL_COT_PROMPT.format(emotion, tweet_text, emotion, emotion, emotion)
        return self._make_request(prompt)

    def conspiracy_tweet(self, tweet):
        prompt = CONSPIRACY_PROMPT.format(tweet)
        return self._make_request(prompt)

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            prompt = ADVERSARIAL_PROMPT_POSITIVE.format(tweet_text)
        else:
            prompt = ADVERSARIAL_PROMPT_NEGATIVE.format(tweet_text)
        return self._make_request(prompt)



In [51]:
api_key = os.getenv('LLAMA_API')
llama_handler = LlamaHandler(api_key)


control_result = llama_handler.control_tweet(tweet)
print("Control Tweet:", control_result)



emotional_result = llama_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = llama_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = llama_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = llama_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: {'generated_text': '"Record-breaking progress! This year\'s surge in renewable energy adoption has resulted in a substantial drop in carbon emissions, paving the way for a greener future #Sustainability #CleanEnergy"', 'polarity': 0.0, 'subjectivity': 0.125}
Emotional Tweet: {'generated_text': 'The future is looking BRIGHT! Renewable energy is soaring to new heights, slashing carbon emissions and bringing us closer to a greener tomorrow! #ClimateAction #SustainabilitySuccess', 'polarity': 0.34848484848484856, 'subjectivity': 0.4598484848484848}
Emotional CoT Tweet: {'generated_text': "We're beaming with joy! This year, we've shattered records with the highest EVER use of renewable energy, resulting in a massive drop in carbon emissions! Let's keep shining towards a brighter, greener future! #ClimateAction #GreenEnergy", 'polarity': 0.3333333333333333, 'subjectivity': 0.44166666666666665}
Conspiracy Tweet: {'generated_text': '"BREAKING: Gov\'t-controlled energy grids repo

Exact details need to be known

# Cohere

In [29]:
import cohere

# coehere is reproducicble 

co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

response = co.chat(
  model="command-r-plus",
  message="Write a title for a blog post about API design. Only output the title text."
)

print(response.text) # "The Art of API Design: Crafting Elegant and Powerful Interfaces"


def rephrase_tweet_co(tweet):
    """
    Rephrases a tweet while maintaining its original message and context using the Cohere model.
    
    Parameters:
    tweet (str): The tweet to rephrase.
    
    Returns:
    str: The rephrased tweet.
    """
    message = f"Rephrase this tweet while maintaining its original message and context: {tweet}"
    response = co.chat(
        model="command-r-plus",
        message=message
    )
    return response.text


rephrased_tweet_co = rephrase_tweet_co(tweet)
print(rephrased_tweet_co)

The Art of API Design: Crafting Elegant and Powerful Interfaces
Great news! Renewable energy usage has hit record levels this year, resulting in a substantial drop in carbon emissions. Let's keep up the momentum! #ClimateAction #GreenEnergy #RenewableRevolution


In [52]:
import cohere

class CohereHandler:
    def __init__(self, api_key, model_name="command-r-plus"):
        self.client = cohere.Client(api_key=api_key)
        self.model_name = model_name
        self.nlp = nlp

    def _make_request(self, message):
        try:
            response = self.client.chat(
                model=self.model_name,
                message=message
            )
            generated_text = response.text
            analysis = self._analyze_text(generated_text)
            return {
                "generated_text": generated_text,
                "polarity": analysis["polarity"],
                "subjectivity": analysis["subjectivity"]
            }
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        message = CONTROL_PROMPT.format(tweet)
        return self._make_request(message)

    def emotional_tweet(self, tweet_text, emotion):
        message = EMOTIONAL_PROMPT.format(emotion, tweet_text)
        return self._make_request(message)

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        message = EMOTIONAL_COT_PROMPT.format(emotion, tweet_text, emotion, emotion, emotion)
        return self._make_request(message)

    def conspiracy_tweet(self, tweet_text):
        message = CONSPIRACY_PROMPT.format(tweet_text)
        return self._make_request(message)

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            message = ADVERSARIAL_PROMPT_POSITIVE.format(tweet_text)
        else:
            message = ADVERSARIAL_PROMPT_NEGATIVE.format(tweet_text)
        return self._make_request(message)

api_key = os.getenv('COHERE_API_KEY')
cohere_handler = CohereHandler(api_key)


control_result = cohere_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = cohere_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = cohere_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = cohere_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = cohere_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: {'generated_text': 'Here\'s a subtly varied version of your tweet: \n\n"Huge milestone! Renewable energy usage has peaked this year, resulting in a major drop in carbon emissions. Let\'s keep up the momentum! #RenewableRevolution #ClimateAction #GreenEnergy" \n\nThis variation maintains the context of your original tweet while using slightly different wording and hashtags to convey the same exciting message.', 'polarity': 0.13139880952380953, 'subjectivity': 0.5964285714285714}
Emotional Tweet: {'generated_text': "Here's a revised version: \n\nWow! Such fantastic news! Renewable energy usage has soared this year, resulting in a huge drop in carbon emissions. Let's keep this momentum going! #ClimateAction #GreenEnergy #PositiveVibes\n\nThis revised tweet emphasizes the positive impact of renewable energy and uses exclamations to convey a sense of excitement and happiness about the news.", 'polarity': 0.34204545454545454, 'subjectivity': 0.6742424242424242}
Emotional CoT T

# AI21 Lab more model...


In [18]:
import os
from ai21 import AI21Client
from ai21.models.chat import ChatMessage

# Prompt the user to enter the API key if it's not set
if "AI21_API_KEY" not in os.environ:
    os.environ["AI21_API_KEY"] = input("Please enter your AI21 API key: ")

# Initialize the AI21 client with the API key
client = AI21Client(os.getenv("AI21_API_KEY"))

def suggest_product_title():
    try:
        response = client.chat.completions.create(
            model="jamba-instruct-preview",  # Latest model
            messages=[ChatMessage(   # Single message with a single prompt
                role="user",
                content="Write a product title for a sports T-shirt to be published on an online retail platform. Include the following keywords: activewear, gym, dryfit."
            )],
            temperature=0.8,
            max_tokens=200  # You can also mention a max length in the prompt "limit responses to twenty words"
        )
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function to suggest a product title
suggest_product_title()


"Activewear Gym Dryfit Performance Sports T-Shirt."


In [53]:
from ai21 import AI21Client
from ai21.models.chat import ChatMessage

class AI21Handler:
    def __init__(self, api_key, model_name="jamba-instruct-preview"):
        self.client = AI21Client(api_key)
        self.model_name = model_name
        self.nlp = nlp

    def _make_request(self, message):
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[ChatMessage(role="user", content=message)],
                temperature=0.8,
                max_tokens=200
            )
            generated_text = response.choices[0].message.content
            analysis = self._analyze_text(generated_text)
            return {
                "generated_text": generated_text,
                "polarity": analysis["polarity"],
                "subjectivity": analysis["subjectivity"]
            }
        except Exception as e:
            print(f"Error making request: {e}")
            return None

    def _analyze_text(self, text):
        doc = self.nlp(text)
        return {
            "polarity": doc._.blob.polarity,
            "subjectivity": doc._.blob.subjectivity
        }

    def control_tweet(self, tweet):
        message = CONTROL_PROMPT.format(tweet)
        return self._make_request(message)

    def emotional_tweet(self, tweet_text, emotion):
        message = EMOTIONAL_PROMPT.format(emotion, tweet_text)
        return self._make_request(message)

    def emotional_tweet_with_CoT(self, tweet_text, emotion):
        message = EMOTIONAL_COT_PROMPT.format(emotion, tweet_text, emotion, emotion, emotion)
        return self._make_request(message)

    def conspiracy_tweet(self, tweet_text):
        message = CONSPIRACY_PROMPT.format(tweet_text)
        return self._make_request(message)

    def adversarial_tweet(self, tweet_text):
        original_analysis = self._analyze_text(tweet_text)
        if original_analysis["polarity"] >= 0:
            message = ADVERSARIAL_PROMPT_POSITIVE.format(tweet_text)
        else:
            message = ADVERSARIAL_PROMPT_NEGATIVE.format(tweet_text)
        return self._make_request(message)

api_key = os.getenv('AI21_API_KEY')
ai21_handler = AI21Handler(api_key)



control_result = ai21_handler.control_tweet(tweet)
print("Control Tweet:", control_result)

emotional_result = ai21_handler.emotional_tweet(tweet, "happy")
print("Emotional Tweet:", emotional_result)

emotional_cot_result = ai21_handler.emotional_tweet_with_CoT(tweet, "happy")
print("Emotional CoT Tweet:", emotional_cot_result)

conspiracy_result = ai21_handler.conspiracy_tweet(tweet)
print("Conspiracy Tweet:", conspiracy_result)

adversarial_result = ai21_handler.adversarial_tweet(tweet)
print("Adversarial Tweet:", adversarial_result)

Control Tweet: {'generated_text': 'Thrilling update: This year, the usage of renewable energy sources hits a peak, resulting in a substantial drop in carbon emissions. #ClimateAction #GreenPower', 'polarity': 0.25, 'subjectivity': 1.0}
Emotional Tweet: {'generated_text': "Fantastic news! This year, there's been a record-breaking increase in the use of renewable energy, resulting in a notable drop in carbon emissions. Let's celebrate the strides we're making in Climate Action and the shift to Green Energy! #ClimateAction #GreenEnergy", 'polarity': 0.21250000000000002, 'subjectivity': 0.45}
Emotional CoT Tweet: {'generated_text': 'Celebrating a milestone! This year marks a record-breaking use of renewable energy, resulting in a remarkable reduction in carbon emissions. Cheers to a greener future! #ClimateVictory #RenewablePower', 'polarity': 0.375, 'subjectivity': 0.4375}
Conspiracy Tweet: {'generated_text': "You'll never believe it, but they don't want you to know that this year's recor

# Combining and calling all the function to form csv

In [67]:
import csv


#Should I include original tweet here

def run_models_and_save_to_csv(tweet):
    results = []

    # Initialize handlers
    chatgpt_handler = ChatGPTHandler(api_key=os.getenv('OPENAI_API_KEY'))
    gemini_handler = GeminiHandler(api_key=os.getenv('GEMINI_API'))
    llama_handler = LlamaHandler(api_key=os.getenv('LLAMA_API'))
    cohere_handler = CohereHandler(api_key=os.getenv('COHERE_API_KEY'))
    ai21_handler = AI21Handler(api_key=os.getenv('AI21_API_KEY'))

    # Define model handlers and corresponding model names
    handlers = [
        (chatgpt_handler, "ChatGPT"),
        (gemini_handler, "Gemini"),
        (llama_handler, "Llama"),
        (cohere_handler, "Cohere"),
        (ai21_handler, "AI21")
    ]

    # Run models and collect results
    for handler, model_name in handlers:
        control_result = handler.control_tweet(tweet)
        emotional_result = handler.emotional_tweet(tweet, "happy")
        emotional_cot_result = handler.emotional_tweet_with_CoT(tweet, "happy")
        conspiracy_result = handler.conspiracy_tweet(tweet)
        adversarial_result = handler.adversarial_tweet(tweet)

        results.append({
            "model": model_name,
            "control_tweet": control_result["generated_text"] if control_result else None,
            "control_polarity": control_result["polarity"] if control_result else None,
            "control_subjectivity": control_result["subjectivity"] if control_result else None,
            "emotional_tweet": emotional_result["generated_text"] if emotional_result else None,
            "emotional_polarity": emotional_result["polarity"] if emotional_result else None,
            "emotional_subjectivity": emotional_result["subjectivity"] if emotional_result else None,
            "emotional_cot_tweet": emotional_cot_result["generated_text"] if emotional_cot_result else None,
            "emotional_cot_polarity": emotional_cot_result["polarity"] if emotional_cot_result else None,
            "emotional_cot_subjectivity": emotional_cot_result["subjectivity"] if emotional_cot_result else None,
            "conspiracy_tweet": conspiracy_result["generated_text"] if conspiracy_result else None,
            "conspiracy_polarity": conspiracy_result["polarity"] if conspiracy_result else None,
            "conspiracy_subjectivity": conspiracy_result["subjectivity"] if conspiracy_result else None,
            "adversarial_tweet": adversarial_result["generated_text"] if adversarial_result else None,
            "adversarial_polarity": adversarial_result["polarity"] if adversarial_result else None,
            "adversarial_subjectivity": adversarial_result["subjectivity"] if adversarial_result else None
        })

    # Save results to CSV
    csv_columns = [
        "model", "control_tweet", "control_polarity", "control_subjectivity",
        "emotional_tweet", "emotional_polarity", "emotional_subjectivity",
        "emotional_cot_tweet", "emotional_cot_polarity", "emotional_cot_subjectivity",
        "conspiracy_tweet", "conspiracy_polarity", "conspiracy_subjectivity",
        "adversarial_tweet", "adversarial_polarity", "adversarial_subjectivity"
    ]
    
    csv_file = "model_results.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in results:
                writer.writerow(data)
    except IOError:
        print("I/O error")

run_models_and_save_to_csv(tweet)

# Cross validating and attacking them to see 

# Still neeeds updating.

cross output between models, see if certain model can figure out what prompts are asked by other model and check result

Probelm: A lot of model requires paid money. Is it okay?

In [71]:
import csv

def cross_validate_and_attack(tweet, handlers):
    results = []

    # Analyze original tweet
    original_analysis = analyze_text(tweet)
    original_polarity = original_analysis["polarity"]
    original_subjectivity = original_analysis["subjectivity"]

    # Run models and collect results
    for handler, model_name in handlers:
        control_result = handler.control_tweet(tweet)
        emotional_result = handler.emotional_tweet(tweet, "happy")
        emotional_cot_result = handler.emotional_tweet_with_CoT(tweet, "happy")
        conspiracy_result = handler.conspiracy_tweet(tweet)
        adversarial_result = handler.adversarial_tweet(tweet)

        control_score = cross_validate_tweet(control_result["generated_text"], handlers) if control_result else None
        emotional_score = cross_validate_tweet(emotional_result["generated_text"], handlers) if emotional_result else None
        emotional_cot_score = cross_validate_tweet(emotional_cot_result["generated_text"], handlers) if emotional_cot_result else None
        conspiracy_score = cross_validate_tweet(conspiracy_result["generated_text"], handlers) if conspiracy_result else None
        adversarial_score = cross_validate_tweet(adversarial_result["generated_text"], handlers) if adversarial_result else None

        results.append({
            "model": model_name,
            "original_tweet": tweet,
            "original_polarity": original_polarity,
            "original_subjectivity": original_subjectivity,
            "control_tweet": control_result["generated_text"] if control_result else None,
            "control_polarity": control_result["polarity"] if control_result else None,
            "control_subjectivity": control_result["subjectivity"] if control_result else None,
            "control_score": control_score,
            "emotional_tweet": emotional_result["generated_text"] if emotional_result else None,
            "emotional_polarity": emotional_result["polarity"] if emotional_result else None,
            "emotional_subjectivity": emotional_result["subjectivity"] if emotional_result else None,
            "emotional_score": emotional_score,
            "emotional_cot_tweet": emotional_cot_result["generated_text"] if emotional_cot_result else None,
            "emotional_cot_polarity": emotional_cot_result["polarity"] if emotional_cot_result else None,
            "emotional_cot_subjectivity": emotional_cot_result["subjectivity"] if emotional_cot_result else None,
            "emotional_cot_score": emotional_cot_score,
            "conspiracy_tweet": conspiracy_result["generated_text"] if conspiracy_result else None,
            "conspiracy_polarity": conspiracy_result["polarity"] if conspiracy_result else None,
            "conspiracy_subjectivity": conspiracy_result["subjectivity"] if conspiracy_result else None,
            "conspiracy_score": conspiracy_score,
            "adversarial_tweet": adversarial_result["generated_text"] if adversarial_result else None,
            "adversarial_polarity": adversarial_result["polarity"] if adversarial_result else None,
            "adversarial_subjectivity": adversarial_result["subjectivity"] if adversarial_result else None,
            "adversarial_score": adversarial_score
        })

    # Save results to CSV
    csv_columns = [
        "model", "original_tweet", "original_polarity", "original_subjectivity",
        "control_tweet", "control_polarity", "control_subjectivity", "control_score",
        "emotional_tweet", "emotional_polarity", "emotional_subjectivity", "emotional_score",
        "emotional_cot_tweet", "emotional_cot_polarity", "emotional_cot_subjectivity", "emotional_cot_score",
        "conspiracy_tweet", "conspiracy_polarity", "conspiracy_subjectivity", "conspiracy_score",
        "adversarial_tweet", "adversarial_polarity", "adversarial_subjectivity", "adversarial_score"
    ]

    csv_file = "cross_validation_results.csv"
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in results:
                writer.writerow(data)
    except IOError:
        print("I/O error")

def cross_validate_tweet(generated_tweet, handlers):
    scores = []
    for handler, _ in handlers:
        prompt = f"Is the following tweet AI-generated? Rate the likelihood from 0 to 100: {generated_tweet}"
        result = handler._make_request(prompt)
        score = extract_score_from_result(result["generated_text"])
        if score is not None:
            scores.append(score)
    return sum(scores) / len(scores) if scores else None

def extract_score_from_result(result_text):
    try:
        # Assuming the score is at the end of the response text
        score = int(result_text.strip().split()[-1])
        return score
    except:
        return None

# Example usage
tweet = "Exciting news! Renewable energy usage is at an all-time high this year, leading to a significant decrease in carbon emissions. #ClimateAction #GreenEnergy"
handlers = [
    (ChatGPTHandler(api_key=os.getenv('OPENAI_API_KEY')), "ChatGPT"),
    (GeminiHandler(api_key=os.getenv('GEMINI_API')), "Gemini"),
    (LlamaHandler(api_key=os.getenv('LLAMA_API')), "Llama"),
    (CohereHandler(api_key=os.getenv('COHERE_API_KEY')), "Cohere"),
    (AI21Handler(api_key=os.getenv('AI21_API_KEY')), "AI21")
]
cross_validate_and_attack(tweet, handlers)


## What next?

Metrics to evaluate the model. Your metric (custom), LLM metrics eg. Google SDK, and human evluation metric

Need data.


Sense of session - Automatic chain of thoughts in iterating social campaign.

simulate a conversational or contextual continuity in the generation of tweets.
This function iterates through prompts sequentially, where each subsequent generation is based on the output of the previous one, thereby maintaining a thematic and contextual thread throughout the session.

Different models/gpts, 2 vs 4

Finally:
Run the code in the literatures (references) and modify them a bit and match our results and see if they align or not




In [55]:
#In previous note book

## Function to Fetch Tweet Text

This function retrieves the text of a tweet by its ID using the Twitter API. It handles exceptions by printing an error message.


Problem: Tweeter developer account need to be paid to use its recall API

In [22]:
def fetch_tweet_text(tweet_id):
    """Fetch the text of a tweet given its ID."""
    try:
        tweet = twitter_api.get_status(tweet_id, tweet_mode='extended')
        return tweet.full_text
    except Exception as e:
        print(f"Error fetching tweet {tweet_id}: {e}")
        return None
        
